In [1]:
from pprint import pprint

## Local Search - Genetic Algorithm

There are some key ideas in the Genetic Algorithm.

First, there is a problem of some kind that either *is* an optimization problem or the solution can be expressed in terms of an optimization problem.
For example, if we wanted to minimize the function

$$f(x) = \sum (x_i - 0.5)^2$$

where $n = 10$.
This *is* an optimization problem. Normally, optimization problems are much, much harder.

![Eggholder](http://www.sfu.ca/~ssurjano/egg.png)!

The function we wish to optimize is often called the **objective function**.
The objective function is closely related to the **fitness** function in the GA.
If we have a **maximization** problem, then we can use the objective function directly as a fitness function.
If we have a **minimization** problem, then we need to convert the objective function into a suitable fitness function, since fitness functions must always mean "more is better".

Second, we need to *encode* candidate solutions using an "alphabet" analogous to G, A, T, C in DNA.
This encoding can be quite abstract.
You saw this in the Self Check.
There a floating point number was encoded as bits, just as in a computer and a sophisticated decoding scheme was then required.

Sometimes, the encoding need not be very complicated at all.
For example, in the real-valued GA, discussed in the Lectures, we could represent 2.73 as....2.73.
This is similarly true for a string matching problem.
We *could* encode "a" as "a", 97, or '01100001'.
And then "hello" would be:

```
["h", "e", "l", "l", "o"]
```

or

```
[104, 101, 108, 108, 111]
```

or

```
0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1
```

In Genetics terminology, this is the **chromosome** of the individual. And if this individual had the **phenotype** "h" for the first character then they would have the **genotype** for "h" (either as "h", 104, or 01101000).

To keep it straight, think **geno**type is **genes** and **pheno**type is **phenomenon**, the actual thing that the genes express.
So while we might encode a number as 10110110 (genotype), the number itself, 182, is what goes into the fitness function.
The environment operates on zebras, not the genes for stripes.

In [2]:
ALPHABET = "abcdefghijklmnopqrstuvwxyz "

### generate_random_candidate
* Generates a random population of candidate genotypes based on the length of the target string and the provided possible genotypes.
* Args:
    * **population_size** (int): The number of candidates to generate.
    * **target_string** (str): The target string used to determine the length of each candidate genotype.
    * **possible_genotypes** (str): A string of possible genotypes to use when generating each candidate.
* Returns:
    * **population** (list): A list containing the randomly generated candidate genotypes.


In [3]:
import random
def generate_random_candidate(length_of_genotype, possible_genotypes):
    random_candidate = []
    for _ in range(length_of_genotype):
        random_character = random.choice(possible_genotypes)
        random_candidate.append(random_character)
    return random_candidate

In [4]:
test_candidate_1 =  generate_random_candidate(3,ALPHABET)
assert  len(test_candidate_1) == 3
for char in test_candidate_1:
    assert char in ALPHABET

### generate_random_population

In [5]:
def generate_random_population(population_size:int, target_string:str, possible_genotypes:list)->list:
    population = []
    target_size = len(target_string)
    for _ in range(population_size):
        candidate = generate_random_candidate(target_size, possible_genotypes)  # Use length of target string for candidate length
        population.append(candidate)
    return population

In [6]:
population_size = 5
target_string = "hello"
possible_genotypes = ["a", "b", "c", "d", "e"]

population = generate_random_population(population_size, target_string, possible_genotypes)
assert len(population) == population_size
for candidate in population:
    for gene in candidate:
        assert gene in possible_genotypes

### calculate_similarity
* Calculates the number of positions where the candidate and target strings have matching characters. This will be the measure for fitness of problem 1, the higher this is the better the candidate
* Args:
    * **candidate** (list): The candidate string to compare.
    * **target** (str): The target string to compare against.
    * **genomes** (any, optional): Additional data; not used in this function but needed for templating the config file. Defaults to `None`.
* Returns:
    * **similarity** (int): The count of matching characters between the candidate and target strings.


In [7]:
def calculate_similarity(candidate, target,genomes=None):
    similarity = 0
    for i in range(len(target)):
        if candidate[i] == target[i]:
            similarity += 1
    return similarity

In [8]:
candidate = ["a", "b", "c"]
target = 'cba'

similarity = calculate_similarity(candidate, target)
assert similarity == 1
candidate = ["c","b","a"]
similarity = calculate_similarity(candidate, target)
assert similarity == 3
candidate = ["x","y","z"]
similarity = calculate_similarity(candidate, target)
assert similarity == 0

### evaluate
* Evaluates the fitness of each candidate in a population against a target using a specified fitness function.
* Args:
    * **population** (list): A list of candidate solutions to be evaluated.
    * **target** (any): The target value or data that candidates are compared against.
    * **fitness_function** (callable): A function that computes the fitness score of a candidate. It should accept arguments `(candidate, target, genomes)`.
    * **genomes** (any, optional): Additional data that may be required by the fitness function. Defaults to `None`.
* Returns:
    * **fitness_scores** (list(tuples)): A list of fitness scores corresponding to each candidate in the population.


In [9]:
def evaluate(population, target, fitness_function:callable, genomes = None):
    fitness_scores = []
    for candidate in population:
        fitness = fitness_function(candidate, target, genomes)
        fitness_scores.append(fitness)
    return fitness_scores

In [10]:
population = [["a", "b", "c"], ["d", "e", "f"], ["g", "b", "i"]]
target = "abc"
fitness_scores = evaluate(population, target, calculate_similarity)
assert len(fitness_scores) == len(population)
assert fitness_scores[0] == 3
assert fitness_scores[1] == 0
assert fitness_scores[2] == 1

### tournament_selection
* Selects the best individual from a randomly chosen subset (tournament) of the population based on their fitness scores. The randomization allows us to explore solution that we would otherwise not check with other algorithms such as hill-climbing
* Args:
    * **population** (list(list)): The list of individuals in the population.
    * **fitness_scores** (list): The list of fitness scores corresponding to each individual in the population.
    * **tournament_size** (int): The number of individuals to include in the tournament.
* Returns:
    * **best_individual** (list): The individual with the highest fitness score among the tournament participants.


In [11]:
def tournament_selection(population, fitness_scores, tournament_size):

    selected_indices = set()  
    selected_individuals = []
    
    while len(selected_individuals) < tournament_size:
        index = random.randint(0, len(population) - 1)
        if index not in selected_indices:
            selected_individuals.append((population[index], fitness_scores[index]))
            selected_indices.add(index)  

    best_individual = selected_individuals[0]

    for individual in selected_individuals:
        if individual[1] > best_individual[1]:
            best_individual = individual

    return best_individual[0]

In [12]:
population = [["a", "b", "c"], ["d", "e", "f"], ["g", "h", "i"]]
fitness_scores = [10, 30, 20] 
tournament_size = 2

selected_individual = tournament_selection(population, fitness_scores, tournament_size)

assert selected_individual == ["d", "e", "f"] or selected_individual == ["g", "h", "i"]
assert selected_individual in population
selected_index = population.index(selected_individual)
assert fitness_scores[selected_index] > min(fitness_scores)

### mutate
* Mutates a candidate by randomly changing characters with a given probability. This helps with not allowing the candidate to fall into a local maximum.

* Args:
    * candidate (list): The candidate to mutate.
    * mutation_probability (float): The probability of mutating each character. Needs to be between 0 and 1
        genomes (str): The string containing possible characters.

* Returns:
        list: The mutated candidate.

In [13]:
def mutate(candidate, mutation_probability, genomes):

    mutated_candidate = []
    for gene in candidate:
        if random.random() < mutation_probability:
            mutated_candidate.append(random.choice(genomes))
        else:
            mutated_candidate.append(gene)
    return mutated_candidate

In [14]:
candidate = ["a", "b", "c", "d", "e"]
mutation_probability = 1.0
genomes = "abcdefghijklmnopqrstuvwxyz"

mutated_candidate = mutate(candidate, mutation_probability, genomes)
assert mutated_candidate != candidate
assert len(mutated_candidate) == len(candidate)

mutation_probability = 0.0
mutated_candidate = mutate(candidate, mutation_probability, genomes)
assert mutated_candidate ==  candidate

### reproduce 
* Performs crossover to reproduce two children from two parents. It will also call mutate to give a chance of mutation to each child


* Args:
  * parent1 (list): The first parent.
  * parent2 (list): The second parent.
  * crossover_probability (float): The probability of performing crossover.

* Returns:
  * tuple: Two children (child1, child2).


In [15]:
def reproduce(parent1, parent2, crossover_probability, mutation_probability, genomes):

    if random.random() < crossover_probability:
        crossover_point = random.randint(0, len(parent1) - 1)
        child1 = parent1[:crossover_point] + parent2[crossover_point:]
        child2 = parent2[:crossover_point] + parent1[crossover_point:]
    else:
        child1, child2 = parent1, parent2  # No crossover, children are clones of the parents

    child1 = mutate(child1,mutation_probability,genomes)
    child2 = mutate(child2,mutation_probability,genomes)
    return child1, child2

In [16]:
parent1 = ["a", "b", "c", "d", "e"]
parent2 = ["v", "w", "x", "y", "z"]
crossover_probability = 1.0 
mutation_probability = 0.0 
genomes = "abcdefghijklmnopqrstuvwxyz"

child1, child2 = reproduce(parent1, parent2, crossover_probability, mutation_probability, genomes)

assert len(child1) == len(parent1)
assert len(child2) == len(parent2)

crossover_probability = 0.0 
mutation_probability = 0.0 
child1, child2 = reproduce(parent1, parent2, crossover_probability, mutation_probability, genomes)
assert child1 == parent1
assert child2 == parent2

### best_global_candidate
* Updates the best fitness score and genotype if a better candidate is found in the current population.
* Args:
    * **best_fitness** (float): The current best fitness score.
    * **best_genotype** (list): The genotype corresponding to the current best fitness score.
    * **population** (list(list)): The list of candidate genotypes in the current population.
    * **fitness_scores** (list): The list of fitness scores corresponding to each genotype in the population.
* Returns:
    * **best_fitness** (float): The updated best fitness score.
    * **best_genotype** (list): The updated genotype corresponding to the best fitness score.


In [17]:
def best_global_candidate(best_fitness, best_genotype, population, fitness_scores):
    max_fitness_score = max(fitness_scores)
    best_index = fitness_scores.index(max_fitness_score)
    current_best_genotype = population[best_index]
    if max_fitness_score > best_fitness:
        best_fitness = max_fitness_score
        best_genotype = current_best_genotype
    
    return best_fitness, best_genotype

In [18]:
best_fitness = 5
best_genotype = ["a", "b", "c"]
population = [["a", "b", "c"], ["d", "e", "f"], ["g", "h", "i"]]
fitness_scores = [5, 6, 4]

new_best_fitness, new_best_genotype = best_global_candidate(best_fitness, best_genotype, population, fitness_scores)

assert new_best_fitness == 6
assert len(new_best_genotype) == 3
assert new_best_genotype == ["d", "e", "f"]


### display_best_candidate
* Displays the best candidate from the current generation, including its fitness score, genotype, and phenotype.
* Args:
    * **generation** (int): The current generation number.
    * **population** (list(list)): The list of candidate genotypes in the population.
    * **fitness_scores** (list): The list of fitness scores corresponding to each genotype in the population.
    * **decoder** (callable, optional): A function to decode the genotype into a phenotype. Defaults to `None`.
    * **genomes** (any, optional): Additional data required by the decoder function. Defaults to `None`.
* Returns:
    * **None**


In [19]:
def display_best_candidate(generation, population, fitness_scores, decoder=None, genomes=None):
    max_fitness_score = max(fitness_scores)
    best_index = fitness_scores.index(max_fitness_score)
    current_best_candidate = population[best_index]
    if decoder is None: 
        print(f"Generation {generation}: Best fitness: {max_fitness_score}, "
              f"Genotype: {current_best_candidate}, Phenotype: {''.join(current_best_candidate)}")
    else:
        decoded_phenotype = decoder(current_best_candidate,genomes)
        print(f"Generation {generation}: Best fitness: {max_fitness_score}, "
              f"Genotype: {current_best_candidate}, Phenotype: {''.join(decoded_phenotype)}")

<a id="genetic_algorithm"></a>
### genetic_algorithm
* Executes the genetic algorithm to find the optimal solution based on the provided configuration parameters.
* Args:
    * **config** (dict): A dictionary containing configuration parameters for the genetic algorithm. Expected keys include:
        * `'target'` (any): The target value or data the algorithm aims to optimize towards.
        * `'genomes'` (str): Possible genotypes or genomic data required for generating the population.
        * `'population_size'` (int): The number of individuals in the population.
        * `'num_generations'` (int): The total number of generations to run the algorithm.
        * `'crossover_probability'` (float): The probability that crossover will occur during reproduction. Should be between 0 and 1
        * `'mutation_probability'` (float): The probability that mutation will occur during reproduction. Should be between 0 and 1
        * `'tournament_size'` (int): The number of individuals participating in each tournament selection.
        * `'fitness_function'` (callable): A function to evaluate the fitness of individuals. Should accept arguments `(candidate, target, genomes)`.
        * `'decode_function'` (callable): A function to decode genotypes into phenotypes for display purposes. Should accept arguments `(genotype, genomes)`.
* Returns:
    * **best_individual** (list): The genotype of the best individual found after running the genetic algorithm.


In [20]:
def genetic_algorithm(config): # add your formal parameterse
    target, genomes = config['target'], config['genomes']
    population_size, num_generations= config['population_size'], config['num_generations']
    crossover_probability, mutation_probability = config['crossover_probability'], config['mutation_probability']
    tournament_size, fitness_function = config['tournament_size'], config['fitness_function'] 
    population, generations = generate_random_population(population_size, target, genomes), 0
    decode_function = config['decode_function']
    best_fitness, best_genotype = -1, None
    while generations < num_generations:
        fitness_scores = evaluate(population, target,fitness_function,genomes)
        best_fitness, best_genotype = best_global_candidate(best_fitness, best_genotype, population, fitness_scores)
        next_population = []
        for _ in range(population_size // 2):
            parent1 = tournament_selection(population, fitness_scores, tournament_size)
            parent2 = tournament_selection(population, fitness_scores, tournament_size)
            child1, child2 = reproduce(parent1, parent2, crossover_probability, mutation_probability, genomes)
            next_population.extend([child1, child2])
        population, generations = next_population, generations + 1
        if generations % 10 == 0:
            display_best_candidate(generations, population,fitness_scores, decode_function,genomes)
    best_index = fitness_scores.index(max(fitness_scores))
    return population[best_index] # return the best individual of the entire run.

## Problem 1

The target is the string "this is so much fun".
The challenge, aside from implementing the basic algorithm, is deriving a fitness function based on "b" - "p" (for example).
The fitness function should come up with a fitness score based on element to element comparisons between target v. phenotype.

In [21]:
target1 = "this is so much fun"

In [22]:
# set up if you need it.
config = {
    'target': target1,
    'genomes': ALPHABET,
    'population_size': 100,
    'num_generations': 500,
    'crossover_probability': 0.6,
    'mutation_probability': 0.02,
    'tournament_size': 5,
    'fitness_function': calculate_similarity,
    'decode_function': None
}

In [23]:
result1 = genetic_algorithm(config) # do what you need to do for your implementation but don't change the lines above or below.

Generation 10: Best fitness: 12, Genotype: ['t', 'h', 'i', 's', 'c', 'i', 's', 'e', 'c', 'v', 'b', 'h', 'u', 'n', 'h', ' ', 'q', 'u', 'n'], Phenotype: thiscisecvbhunh qun
Generation 20: Best fitness: 15, Genotype: ['t', 'h', 'i', 's', 'c', 'i', 's', 'e', 's', 'f', ' ', 'h', 'u', 'n', 'h', ' ', 'f', 'u', 'n'], Phenotype: thiscisesf hunh fun
Generation 30: Best fitness: 17, Genotype: ['t', 'h', 'i', 's', ' ', 'i', 's', ' ', 's', 'f', ' ', 'm', 'u', 'n', 'h', ' ', 'f', 'u', 'n'], Phenotype: this is sf munh fun
Generation 40: Best fitness: 19, Genotype: ['t', 'h', 'i', 's', ' ', 'i', 's', ' ', 's', 'o', 'r', 'm', 'u', 'n', 'h', ' ', 'f', 'u', 'n'], Phenotype: this is sormunh fun
Generation 50: Best fitness: 19, Genotype: ['t', 'h', 'i', 's', ' ', 'i', 's', ' ', 's', 'o', ' ', 'm', 'u', 'c', 'h', ' ', 'f', 'u', 'n'], Phenotype: this is so much fun
Generation 60: Best fitness: 19, Genotype: ['t', 'h', 'i', 's', ' ', 'i', 's', ' ', 's', 'o', ' ', 'm', 'u', 'c', 'h', ' ', 'f', 'u', 'n'], Pheno

In [24]:
pprint(result1, compact=True)

['t', 'h', 'i', 's', ' ', 'i', 's', ' ', 's', 'o', ' ', 'm', 'u', 'c', 'h', ' ',
 'f', 'u', 'n']


### calculate_similarity_reverse
* Calculates the number of positions where the candidate string matches the target string in reverse order.
* Args:
    * **candidate** (list): The candidate string to compare.
    * **target** (str): The target string whose reverse will be compared against the candidate.
    * **genomes** (any, optional): Additional data; not used in this function. Defaults to `None`.
* Returns:
    * **fitness** (int): The count of matching characters between the candidate and the reversed inspected target string.


In [25]:
def calculate_similarity_reverse(candidate, target,genomes=None):

    fitness = 0
    target_length = len(target)
    
    for i in range(target_length):
        if candidate[i] == target[target_length - 1 - i]:
            fitness += 1

    return fitness

In [26]:
candidate = "abcde"
target = "edcba"
fitness = calculate_similarity_reverse(candidate, target)
assert isinstance(fitness, int)
assert fitness == 5

candidate = "abcde"
target = "vwxyz"
fitness = calculate_similarity_reverse(candidate, target)
assert fitness == 0


In [27]:
target2 = "nuf hcum os si siht"

In [28]:
# set up if you need it.
# set up if you need it.
config2 = {
    'target': target2,
    'genomes': ALPHABET,
    'population_size': 100,
    'num_generations': 500,
    'crossover_probability': 0.6,
    'mutation_probability': 0.02,
    'tournament_size': 5,
    'fitness_function': calculate_similarity_reverse,
    'decode_function':None
}

In [29]:
result2 = genetic_algorithm(config2) # do what you need to do for your implementation but don't change the lines above or below.

Generation 10: Best fitness: 9, Genotype: ['q', 's', 'i', 's', 'n', 'i', 'z', ' ', 's', 'o', 'r', 'b', 'a', 'z', 'h', 'v', 'r', 'w', 'n'], Phenotype: qsisniz sorbazhvrwn
Generation 20: Best fitness: 13, Genotype: ['b', 'h', 'i', 's', 'w', 'i', 'j', ' ', 's', 'u', ' ', 'm', 'u', 'n', 'g', ' ', 'f', 'n', 'n'], Phenotype: bhiswij su mung fnn
Generation 30: Best fitness: 16, Genotype: ['q', 'h', 'i', 's', ' ', 'i', 'z', ' ', 's', 'o', ' ', 'm', 'u', 'n', 'h', ' ', 'f', 'u', 'n'], Phenotype: qhis iz so munh fun
Generation 40: Best fitness: 18, Genotype: ['t', 'h', 'i', 's', ' ', 'i', 's', ' ', 's', 'o', ' ', 'm', 'u', 'n', 'h', ' ', 's', 'u', 'n'], Phenotype: this is so munh sun
Generation 50: Best fitness: 19, Genotype: ['t', 'h', 'i', 's', ' ', 'i', 's', ' ', 's', 'o', ' ', 'm', 'u', 'c', 'h', ' ', 'f', 'u', 'n'], Phenotype: this is so much fun
Generation 60: Best fitness: 19, Genotype: ['t', 'h', 'i', 's', ' ', 'i', 's', ' ', 's', 'o', ' ', 'm', 'u', 'c', 'h', ' ', 'f', 'u', 'n'], Phenot

In [30]:
pprint(result2, compact=True)

['t', 'h', 'i', 's', ' ', 'i', 's', ' ', 's', 'o', ' ', 'm', 'u', 'c', 'h', ' ',
 'f', 'u', 'n']


In [31]:
ALPHABET3 = "abcdefghijklmnopqrstuvwxyz"

In [32]:
target3 = "guvfvffbzhpusha"

### calculate_caesar_cipher
* Calculates the fitness score by comparing the candidate string to the target string encoded with a Caesar cipher shifted by 13 positions.
* Args:
    * **candidate** (list): The candidate string to evaluate.
    * **target** (str): The target string to be encoded using the Caesar cipher.
    * **genomes** (str): A string of possible genotypes (characters) used for encoding and decoding.
* Returns:
    * **fitness** (int): The number of positions where the candidate matches the encoded target string.


In [33]:
def calculate_caesar_cipher(candidate,target,genomes):
    fitness = 0
    target_length = len(target)
    genome_length = len(genomes)
    for i in range(target_length):
        candidate_gene = candidate[i]
        target_gene = target[i]
        
        if target_gene in genomes:
            target_index = genomes.index(target_gene)
            decoded_index = (target_index + 13) % genome_length
            decoded_char = genomes[decoded_index]
        else:
            continue

        if candidate_gene == decoded_char:
            fitness += 1

    return fitness

### decode_ceasar_cipher
* Decodes a Ceasar cipher encoded string by shifting each character back by 13 positions in the provided genomes string.
* Args:
    * **candidate** (list): The encoded string to be decoded.
    * **genomes** (str): A list of possible characters used for encoding and decoding.
* Returns:
    * **decoded_phenotype** (str): The decoded string after applying the reverse Caesar cipher.


In [34]:
def decode_ceasar_cipher(candidate, genomes):
    decoded_phenotype = []
    for char in candidate:
        true_char_index = (genomes.index(char) - 13) % len(genomes)
        decoded_phenotype.append(genomes[true_char_index])
    return ''.join(decoded_phenotype)


In [35]:
candidate = "uryyb"
genomes = "abcdefghijklmnopqrstuvwxyz"
decoded_message = decode_ceasar_cipher(candidate, genomes)
assert len(decoded_message) == 5
assert decoded_message == "hello"
candidate = "abcdefghijklmnopqrstuvwxyz"
genomes = "abcdefghijklmnopqrstuvwxyz"
decoded_message = decode_ceasar_cipher(candidate, genomes)
assert decoded_message == "nopqrstuvwxyzabcdefghijklm"


In [36]:
# set up if you need it
config3 = {
    'target': target3,
    'genomes': ALPHABET3,
    'population_size': 100,
    'num_generations': 500,
    'crossover_probability': 0.6,
    'mutation_probability': 0.02,
    'tournament_size': 5,
    'fitness_function': calculate_caesar_cipher,
    'decode_function':decode_ceasar_cipher
}

In [37]:
result3 = genetic_algorithm(config3) # do what you need to do for your implementation but don't change the lines above or below.

Generation 10: Best fitness: 9, Genotype: ['f', 'h', 'l', 's', 'z', 'd', 'p', 'o', 'm', 'u', 'c', 'q', 'b', 'u', 'n'], Phenotype: suyfmqcbzhpdoha
Generation 20: Best fitness: 11, Genotype: ['t', 'h', 'l', 's', 'r', 's', 's', 'o', 'm', 'u', 'c', 'q', 'b', 'u', 'n'], Phenotype: guyfeffbzhpdoha
Generation 30: Best fitness: 13, Genotype: ['t', 'h', 'i', 's', 'r', 's', 's', 'o', 'm', 'u', 'c', 'h', 'b', 'u', 'n'], Phenotype: guvfeffbzhpuoha
Generation 40: Best fitness: 14, Genotype: ['t', 'g', 'i', 's', 'r', 's', 's', 'o', 'm', 'u', 'c', 'h', 'b', 'u', 'n'], Phenotype: gtvfeffbzhpuoha
Generation 50: Best fitness: 14, Genotype: ['t', 'h', 'i', 's', 'i', 's', 's', 'o', 'm', 'u', 'c', 'h', 'b', 'u', 'n'], Phenotype: guvfvffbzhpuoha
Generation 60: Best fitness: 14, Genotype: ['t', 'h', 'i', 's', 'i', 's', 's', 'o', 'm', 'u', 'c', 'h', 'w', 'u', 'n'], Phenotype: guvfvffbzhpujha
Generation 70: Best fitness: 15, Genotype: ['t', 'h', 'i', 's', 'i', 's', 's', 'o', 'm', 'u', 'c', 'h', 'f', 'u', 'n'],

In [38]:
pprint(result3, compact=True)

['t', 'h', 'i', 's', 'i', 's', 's', 'o', 'm', 'u', 'c', 'h', 'f', 'u', 'n']


#### I would tackle this in a combination of small steps.
1. Assuming that the spaces in the message are not meant to be decipher, meaning it is just separating the actual strings, I would use them to create a list of list where we can actually evaluate each string individually but the candidate is still the full sentence.
2. One part of the main problem for this is finding the actual shift that is encoding the strings, so this means that we are kind of trying to find the correct translation for the chromosomes, in this case it can be any integer from 0 to 25. 
3. The second part the main problem is to actually be able to define a fitness function that makes sense. The one that comes to my mind is to have a large dictionary with actual english words in it(or whatever languate you are trying to use, this might work with anything in reality). This way we can compare if the decoded word is in this dictionary, and if it is, it will add to the fitness of the candidate.
4. Even if this would help find the actual shifting that is needed, it does not neccesarily mean that the sentence would make sense, just that we are getting actual english words.  